In [60]:
import torch
import matplotlib.pyplot as plt
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torch.nn import functional as F
from torchvision import transforms

In [125]:
transform = transforms.Compose([transforms.Resize((512,512)),transforms.ToTensor()])

In [126]:
training_data = datasets.Flowers102(root = 'data', split = 'train', download = True, transform = transform)
testing_data = datasets.Flowers102(root = 'data', split = 'test', download = True, transform = transform)

In [127]:
trainingLoader = DataLoader(training_data, batch_size=16)
testingLoader = DataLoader(testing_data, batch_size=16)

In [211]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,18,5),
            nn.BatchNorm2d(18),
            nn.ReLU(),
            nn.MaxPool2d(3,3)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(18, 48,5),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.MaxPool2d(3,3)
        )
        self.fc1 = nn.Linear(6912,2000)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(2000,500)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(500,102)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.reshape(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

In [212]:
network = NeuralNetwork()

In [213]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr = 0.01)

In [220]:
epoch = 50

for i in range(epoch):
    running_loss = 0
    for j, data in enumerate(trainingLoader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        
        outputs = network(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if j % 50 == 49:
            print(f'[{i + 1}, {j + 1:5d}] loss: {running_loss/50:.3f}')
            running_loss = 0

print('Finished Training')

[1,    50] loss: 2.498
[2,    50] loss: 2.339
[3,    50] loss: 2.123
[4,    50] loss: 1.915
[5,    50] loss: 1.713
[6,    50] loss: 1.518
[7,    50] loss: 1.339
[8,    50] loss: 1.164
[9,    50] loss: 1.005
[10,    50] loss: 0.864
[11,    50] loss: 0.750
[12,    50] loss: 0.617
[13,    50] loss: 0.489
[14,    50] loss: 0.407
[15,    50] loss: 0.317
[16,    50] loss: 0.252
[17,    50] loss: 0.195
[18,    50] loss: 0.391
[19,    50] loss: 0.360
[20,    50] loss: 0.169
[21,    50] loss: 0.106
[22,    50] loss: 0.082
[23,    50] loss: 0.073
[24,    50] loss: 0.060
[25,    50] loss: 0.052
[26,    50] loss: 0.046
[27,    50] loss: 0.041
[28,    50] loss: 0.038
[29,    50] loss: 0.035
[30,    50] loss: 0.033
[31,    50] loss: 0.031
[32,    50] loss: 0.029
[33,    50] loss: 0.027
[34,    50] loss: 0.025
[35,    50] loss: 0.024
[36,    50] loss: 0.022
[37,    50] loss: 0.021
[38,    50] loss: 0.020
[39,    50] loss: 0.019
[40,    50] loss: 0.019
[41,    50] loss: 0.018
[42,    50] loss: 0.018
[

In [221]:
correct = 0
total = 0

for data in testingLoader:
    images, labels = data
    output = network(images)
    _, predicted = torch.max(output.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    
print(f"Network accuracy = {100 * (correct / total)}%")

Network accuracy = 10.928606277443487%
